### Workflow to Validate NISAR L2Permafrost Displacement Requirement

**Code authored by:** Andrew Johnson, Simon Zwieback, Franz Meyer, Jie Chen</br>

<hr/>

### Table of Contents


[**Prep A. Environment Setup**](#permafrost_prep_a)

[**Prep B. Data Staging**](#permafrost_prep_b)

[**1. Generation of Interferogram stack**](#permafrost_infgs)

[**2. Generation of time series from interferograms**](#permafrost_mintpy)
- [2.1 Set up MintPy configuration file](#permafrost_mintpy_config)
- [2.2 Load data into MintPy](#permafrost_mintpy_load)
- [2.3 Validate/Modify interferogram network](#permafrost_mintpy_validate)
- [2.4 Reference interferograms to common lat/lon](#permafrost_mintpy_reference)
- [2.5 Invert for SBAS line-of-site timeseries](#permafrost_mintpy_invert)

[**3. Optional Corrections**](#permafrost_correct)
- [3.1 Troposphere correction](#permafrost_corect_troposphere)
- [3.2 Phase deramping](#permafrost_correct_phase)
- [3.3 Topographic residual correction](#permafrost_correct_topography)

[**4. Validation method A: InSAR-only struction functions**](#permafrost_insar_validate)
- [4.1 Use structure functions to identify pixel pairs](#permafrost_insar_structure)
- [4.2 Evaluate displacements of pairs with respect to requirement](#permafrost_insar_evaluate)

[**5. Validation method B: Comparison to ground truth displacements**](#permafrost_field_validate)
- [5.1 Load field data](#permafrost_field_data)
- [5.2 Prepare InSAR for field comparison](#permafrost_field_prepare)
- [5.3 Re-run MintPy with field reference points](#permafrost_reference_point)
- [5.4 Evaluate displacements with respect to requirement](#permafrost_field_evaluate)

<hr/>
<a id='permaforst_prep_a'></a>

### Environment Setup

#### Load Python Packages

In [ ]:
import numpy as np
import pandas as pd
import json
import os
import subprocess
import h5py

from datetime import datetime as dt
from matplotlib import pyplot as plt
from pathlib import Path
from mintpy import plot_network,view
from mintpy.utils import readfile
from solid_utils import permafrost_utils as pu
from solid_utils.sampling import load_geo_utm, samp_pair,load_geo_utm
from solid_utils.plotting import display_permafrost_validation 
from solid_utils.plotting import display_validation_table
from solid_utils.saving import save_results

In [ ]:
site = 'NorthSlopeEastD102'
requirement = 'permafrost'
dataset = 'Hyp3_S1'
year = 2025
start_directory = 'default'

custom_sites = "/home/jovyan/my_sites.txt"  # Path to custom site metadata
try:
    with open(custom_sites, "r") as f:
        sitedata = json.load(f)
    site_info = sitedata["sites"][site]
except (FileNotFoundError, json.JSONDecodeError) as e:
    raise RuntimeError(f"Failed to load site metadata from {custom_sites}: {e}")
except KeyError:
    raise ValueError(f"Site ID '{site}' not found in {custom_sites}")

In [ ]:
################# Set Directories ##########################################
print('\nCurrent directory:',os.getcwd())

start_directory = Path(f'/scratch/nisar-st-calval-solidearth/permafrost')
work_dir = os.path.join(start_directory,dataset,site,str(year))
field_dir = os.path.join(start_directory,'fielddata')

gunw_dir = os.path.join(work_dir,'products')
mintpy_dir = os.path.join(work_dir,'MintPy')
os.makedirs(mintpy_dir,exist_ok=True)
print("   MintPy  dir:", mintpy_dir)

# # Change to Workdir   
os.chdir(mintpy_dir)

<a id='permafrost_infgs'></a>
# 1. Generation of interferogram stack

The NISAR project will provide sets of fully coregistered ascending and descending unwrapped L2 interferograms (aka InSAR “stacks”) over regions of interest listed in the NISAR Solid Earth calval document. Prior to the launch of NISAR, we use interferograms generated by the Hybrid Pluggable Processing Pipeline (Hyp3) at the Alaska Satellite Facility (ASF) from Sentinel-1 SAR data. Hyp3 interfergram generation uses GAMMA and identifies overlapping Sentinel-1 bursts, and coregisters them using an iterative offset polynomial method and an Enhanced Spectral Diversity algorithm. Information about the Hyp3 inferferogram generation [can be found here](#https://hyp3-docs.asf.alaska.edu/guides/insar_product_guide/).

Use the [Generating Hyp3 Interferograms](./Generate_Hyp3_Interferograms.ipynb) notebook to produce and download this data. We create interferogram stacks to cover a 96 day summer time period over on permafrost terrain to measure displacement when the surface is snow-free and traditionally experiencing subsidence.

<a id='permafrost_mintpy'></a>
# 2. Generation of time series from interferograms

InSAR time series (i.e. the unfiltered displacement of each pixel vs. time) will be estimated from a processed InSAR stack using a variant of the small baseline subset (SBAS) approach. For this step, we use a time series inversion method of the Miami InSAR Time-series software in Python (MintPy) with 12 and 24 day interferogram pairs.

Note: this code is currently set up to run on MintPy version 1.5.1.

In [ ]:
# Loading data into mintpy, if not done already
config_file = os.path.join(mintpy_dir,site + f'_{year}.cfg')
command = 'smallbaselineApp.py ' + str(config_file) + ' --dostep load_data'
process = subprocess.run(command, shell=True)

<a id='permafrost_mintpy_validate'></a>
## 2.1 Validate/Modify Interferogram Network


In [ ]:
config_file = os.path.join(mintpy_dir,site + f'_{year}.cfg')
command = 'smallbaselineApp.py ' + str(config_file) + ' --dostep modify_network'
process = subprocess.run(command, shell=True)

<a id='permafrost_mintpy_reference'></a>
## 2.2 Reference interferograms to common Lat/Lon

Note: The printed ```REF_LAT``` and ```REF_LON``` from this step will be the UTM y and x coordinates, respectively. ```REF_X``` and ```REF_Y``` are the j and i (column and row) indices of the reference.

In [ ]:
command = 'smallbaselineApp.py ' + str(config_file) + ' --dostep reference_point'
process = subprocess.run(command, stdout=subprocess.PIPE, stderr=subprocess.STDOUT, text=True, shell=True)
os.system('info.py inputs/ifgramStack.h5 | egrep "REF_"');

<a id='permafrost_mintpy_invert'></a>
## 2.3 Invert for SBAS Line-of-Sight Timeseries

Run the network inversion to generate the timeseries file.

Note: If run in OpenScienceLab, this step takes approximately 30 minutes to complete.

In [ ]:
command = 'smallbaselineApp.py ' + str(config_file) + ' --dostep invert_network'
process = subprocess.run(command, shell=True)

<a id='permafrost_correct'></a>
# 3. Optional corrections

Mintpy provides functionality for a number of standard corrections which might improve the accuracy of the timeseries inversion or displacement results. Several corrections are shown here:

* Troposphere correction </br>
* Deramping </br>
* Topographic residual correction

None of these corrections are currently applied in the analysis presented in this notebook, but the code here can easily be modified to add this functionality if desired.

<a id='permafrost_correct_troposphere'></a>
## 3.1 Trophosphere correction

Atmospheric effects, especially moisture, can cause phase delays in the measured satellite signal. ERA5 reanalysis products provide information about atmospheric moisture and can be used to estimate the tropospheric phase delays. Performing this correction requires obtaining the ERA5 reanalysis products separately.

In [ ]:
do_tropo_correction = False
########################################################################
'''
REFERENCE : https://github.com/insarlab/pyaps#2-account-setup-for-era5
Read Section 2 for ERA5 [link above] to create an account on the CDS website.
'''

if do_tropo_correction:
    if not Use_Staged_Data and not os.path.exists(Path.home()/'.cdsapirc'):
        print('NEEDED to download ERA5, \
        link: https://cds.climate.copernicus.eu/user/register')
        UID = input('Please type your CDS_UID:')
        CDS_API = input('Please type your CDS_API:')
        
        cds_tmp = '''url: https://cds.climate.copernicus.eu/api/v2
        key: {UID}:{CDS_API}'''.format(UID=UID, CDS_API=CDS_API)
        os.system('echo "{cds_tmp}" > ~/.cdsapirc; chmod 600 ~/.cdsapirc'
            .format(cds_tmp = str(cds_tmp)))
    
    command = 'smallbaselineApp.py ' + str(config_file) + ' --dostep \
    correct_troposphere'
    process = subprocess.run(command, shell=True)
    
    # view.main(['inputs/ERA5.h5'])
    timeseries_filename = 'timeseries_ERA5.h5'
else:
    timeseries_filename = 'timeseries.h5'

<div class="alert alert-block alert-info">
    <b>Note:</b> The rest of the notebook is using the variable <code>timeseries_filename</code> as InSAR result. <b>Make sure <code>timeseries_filename</code> is set to the correct file</b>
</div>

<a id='permafrost_correct_phase'></a>
## 3.2 Phase deramping
\[Mintpy provides this functionality]

In [ ]:
command = 'smallbaselineApp.py ' + str(config_file) + ' --dostep deramp'
process = subprocess.run(command, shell=True)

<a id='permafrost_correct_topography'></a>
## 3.3 Topographic residual correction
\[Mintpy provides this functionality\]

In [ ]:
command = 'smallbaselineApp.py ' + str(config_file) + ' --dostep correct_topography'
process = subprocess.run(command, shell=True)

<!-- <a id='permafrost_insar_validate'></a>
# 4. Validation Method 1: Comparison to observed field data

The NISAR permafrost validation requirement states that at least 80% of the time, the difference in surface displacement for two given points over 90 days should be no greater than $4(1+\sqrt{\text{L}})$ mm for points of L km apart, with $\text{L} \leq 50$ km.
 -->

<a id='permafrost_field_validate'></a>
# 4. Validation Method 1: Comparison to ground truth displacements

![Survey at Happy Valley East](HappyValleyEast_level.png)
Photo: Level survey being conducted at Happy Valley East in Sept., 2025. Credit: Andrew Johnson




<!-- We can compare the Interferometric displacements to the measured displacements at the field sites. The locations of the field sites are as follows (XY values in UTM, EPSG 32605):

Happy Valley (HV): x=664946, y=7677215</br>
Happy Valley East (HVE): x=665175, y=7677290</br>
Ice Cut (IC): x=666290, y=7664601</br>
Slope Mountain (SM): x=664156, y=7629999</br> -->

In [ ]:
pointnames = ['HV','HVE','IC','SM']
pointlocs = [[1114,3285],[1113,3288],[1272,3302],[1705,3275]]
pointlocsxy = [[664946,7677215],[665175,7677290],[666290,7664601],[664156,7629999]]
pointsubsetlocs = [[119, 287], [118, 290], [277, 304], [710, 277]]

<a id='permafrost_field_data'></a>
## 4.1 Load field data

Field observations of summer subsidence were taken for several areas of interest on the North Slope of Alaska, along the Dalton Highway. The ground in this area is snow-free for approximately three months each summer, usually from late May to late August. Measurements from leveling (surveying) and GNSS were taken at the beginning and end of this summer time period. Each field area of interest has 153 sampled points over a 100 m square on the ground.

The end result of these observations are direct measurements of the surface subsidence which can be compared with the satellite-InSAR derived surface displacement product. See the accompanying notebook [Permafrost_fielddata](./Permafrost_fielddata.ipynb) for details. That notebook produces a small file called ```field_results.csv```, which is an input for this validation.

We create an array for the relative displacement of each field site relative to each other site.

In [ ]:
#Load in observed data
fielddata_file = Path(work_dir)/'field_results.csv'

obsdata = pd.read_csv(fielddata_file) #ground truth observations
obsdisp = obsdata['rel_change'][:]
obsstd = obsdata['stdev'][:]
obsdiff = np.zeros((len(obsdisp),len(obsdisp)))
obsdiffstd = np.zeros_like(obsdiff)
fielddates = []
for i in range(len(obsdisp)):
    for j in range(len(obsdisp)):
        obsdiff[i,j] = obsdisp[i]-obsdisp[j]

        #use sum of squares to propagate error of subtracted measurements
        obsdiffstd[i,j] = np.sqrt(obsstd[i]**2+obsstd[j]**2)
    
    date1 = dt.strptime(str(obsdata['date1'][i]),'%Y-%m-%d')
    date2 = dt.strptime(str(obsdata['date2'][i]),'%Y-%m-%d')
    fielddates.append([date1,date2])

<a id='permafrost_field_prepare'></a>
## 4.2 Prepare InSAR for field comparison


In order to perform double differencing of InSAR displacements in a way that will be comparable to the field observations, we run the mintpy inversion several times, with the reference point set to a different field validation site each time, and also using a spline fitting function to fit the InSAR timeseries to the exact dates of the field measurements.


### 4.2.1 Perform timeseries reconstruction

The displacement time series from mintpy is reconstruted here by spline-fitting it the time series to a set of basis functions, following Zwieback et al. (2020). The basis functions represent typical summer permafrost surface subsidence patterns, due melting ice in the active layer. The spline fitting reconstruction both reduces the noise from individual acquisitions, and also allows us to identify the InSAR displacement for arbitrary dates within the period.


In [ ]:
ts = pu.Timeseries.from_file(fn = Path(mintpy_dir)/timeseries_filename)
sm = pu.SplineModel(dates_o=ts.dates)

#perform the reconstruction
ts_rec = sm.reconstruct(timeseries=ts)

# plot basis functions
pu.plot_basis_functions(sm)

Now we will demonstrate the reconstuction at a single point.

In [ ]:
ipt,jpt = 500,200

plt.figure()
plt.axhline(y=0,linestyle='--',color='k')
plt.plot(ts.dates,ts.timeseries[:,ipt,jpt],'.',markersize=12,label='data')
plt.plot(ts_rec.dates,ts_rec.timeseries[:,ipt,jpt],'.-',
         markersize=8,label='reconstruction')
plt.xticks(rotation=30)
plt.legend()

Additionally we can show the summer time displacement across the entire region:

In [ ]:
#The fieldsites of Happy Valley and Happy Valley East are too close to plot separately in this step.
labelpoints = [[119, 287], [277, 304], [710, 277]]
labelpointnames = ['HV/HVE','IC','SM']

plt.figure()
plt.imshow(ts_rec.timeseries[-1]-ts_rec.timeseries[0],
           vmin=-0.10,vmax=0.10,cmap='RdBu')
plt.colorbar()
for k,name in enumerate(labelpointnames):
    pt = labelpoints[k]
    plt.plot(pt[1],pt[0],'*',markersize=8,color='k')
    plt.text(pt[1]+20,pt[0]+10,name,fontsize=12)


<a id='permafrost_insar_diff'></a>
### 4.3 Compare InSAR and field displacement results

First we compare the InSAR displacement between the field sites for the given field observations dates

In [ ]:
#Retrieve InSAR displacment differences between points
locdisp = []
for k,name in enumerate(pointnames):
    ts_rec = sm.reconstruct(timeseries=ts,dates_r=fielddates[k])
    ipt,jpt = pointsubsetlocs[k]
    locdisp.append(ts_rec.timeseries[-1,ipt,jpt])

Next we compare the double difference of displacement between each subsite pair from InSAR and field data

In [ ]:
__,atrib = readfile.read(Path(mintpy_dir)/timeseries_filename) 
xst = float(atrib['X_STEP'])
yst = float(atrib['Y_STEP'])

dispdiff = []
obsdiff = []
obserr = []
ddf = []
dist = []

locvec = locdisp
loclen = len(locdisp)

for k in range(loclen):
    iref,jref = pointsubsetlocs[k]
    for l in range(k+1,loclen):    
        ipt,jpt = pointsubsetlocs[l]
        dispdiff.append(locvec[k]-locvec[l]) #insar displacement difference
        obsdiff.append(obsdisp[k]-obsdisp[l]) #field data displacement difference
        obserr.append(np.sqrt(obsstd[k]**2+obsstd[l]**2))
        ddf.append(np.abs(np.abs(dispdiff[-1])-np.abs(obsdiff[-1])))
        dist.append(np.sqrt(((iref-ipt)*yst)**2+((jref-jpt)*xst)**2)/1e3)

<a id='permafrost_field_evaluate'></a> 
## 4.4 Evaluate displacements to NISAR permafrost requirement

### 4.4.1 Compute requirement validation evaluation

In [ ]:
ddf_cm = np.array(ddf)*100               #double difference in cm
obserr_cm = np.array(obserr)*100         #observation error in cm
dist = np.array(dist)                 #distance (in km)
reqvec = 4*(1+np.sqrt(dist))/10       #requirement in cm

#remove HVE in 2023, and SM in 2024
sitemask = np.array([1,1,1,1,1,1],dtype='bool')
if year == 2023:
    sitemask = np.array([0, 1, 1, 0, 0, 1],dtype='bool')
if year == 2024:
    sitemask = np.array([1, 1, 0, 1, 0, 0],dtype='bool')
ddf_cm = ddf_cm[sitemask]
obserr_cm = obserr_cm[sitemask]
dist = dist[sitemask]
reqvec = reqvec[sitemask]

succvec = np.array(ddf_cm <= reqvec)
succerr = np.array(ddf_cm-obserr_cm <= reqvec)
req_met_rate = np.sum(succvec)/len(succvec)
req_within_MOE = np.sum(succerr)/len(succerr)

## 4.4.2 Visualize validation results

In [ ]:
import matplotlib.patches as patches

req_thresh = 0.8
val, val_MOE = False,False
val_str, val_MOE_str = 'FAILED','FAILED'
if req_met_rate>=req_thresh:
    val = True
    val_str = 'PASSED'
if req_within_MOE >= req_thresh:
    val_MOE = True
    val_MOE_str='PASSED'


print(f"Percent of double differences which meet the requirement: \
{req_met_rate*100:.1f}%, validation: {val_str}")
print(f"Percent of double differences within margin of error of requirement: \
{req_within_MOE*100:.1f}%, validation: {val_MOE_str}")

dist_th = np.linspace(0.1,50,100)  # distances for evaluation
acpt_error = 4*(1+np.sqrt(dist_th))  # permafrost threshold in mm
acpt_error_cm = acpt_error/10.
start_date = np.min(fielddates)
end_date = np.max(fielddates)

fig, ax = plt.subplots(figsize=(11,7))
plt.scatter(dist,ddf_cm,label='|$D_{obs} - D_{InSAR}|$ for pair')
plt.plot(dist_th, acpt_error_cm, 'r',label='Permafrost requirement')
# ax.set_xscale('log')
plt.ylim(0,7)
plt.xlim(0,50)
plt.legend(loc='upper left')
plt.title(f"Double-Difference Residuals \n Date range \
{start_date.strftime('%Y-%m-%d')} to {end_date.strftime('%Y-%m-%d')} \n InSAR\
and surveying/GNSS simple subtraction displacement")
plt.xlabel("Distance (km)")
plt.ylabel("Amplitude of Double-Differenced Displacement Residual (cm)")
plt.errorbar(dist,ddf_cm,yerr=obserr_cm,ls='none',capsize=5)


# Add legend with validation info 
textstr = f'Permafrost requirement\n'
textstr += f'Site: NorthSlopeEast\n'
# if validation.loc['Total']['success_fail']:
if val:
    validation_color = '#239d23'
else:
    validation_color = '#bc2e2e'


props = {**{'facecolor':'none', 'edgecolor':'none'}}
ax.text(37, 6.15, textstr, fontsize=12)#, bbox=props)
ax.text(39, 6.05,  val_str, fontsize=16, weight='bold')

rect = patches.Rectangle((36.5, 5.95), 12.5, 0.9,
                       linewidth=1, edgecolor='black',
                       facecolor=validation_color)
ax.add_patch(rect)

plt.show()


### 4.4.3 Prepare validation table

In [ ]:
# validation
req_thresh = 0.8

count = np.array([np.size(i) for i in ddf_cm])
passed = np.array([np.sum(i) for i in succvec])
passed_within_MOE = np.array([np.sum(i) for i in succerr])
success = passed/count >= req_thresh
success_within_MOE = passed_within_MOE/count >= req_thresh

columns = ['distance[km]', 'count', 'passed',
           'within_MOE', 'success', 'success_within_MOE']

validation = pd.DataFrame(np.vstack([dist,
                                     count,
                                     passed,
                                     passed_within_MOE,
                                     success,
                                     success_within_MOE],dtype=object).T,
                                     columns = columns)

sumrow = ['Total',
           np.sum(count),
           np.sum(passed),
           np.sum(passed_within_MOE),
           np.sum(passed)/np.sum(count) >= req_thresh,
           np.sum(passed_within_MOE)/np.sum(count) >= req_thresh]
validation = pd.concat([validation, pd.DataFrame([sumrow],columns = columns)],ignore_index=True)
if validation.loc[validation.index[-1]]['success']:
    print("This displacement dataset passes the requirement.")
else:
    print("This displacement dataset does not pass the requirement.")
if validation.loc[validation.index[-1]]['success_within_MOE']:
    print("This displacement dataset passes the requirement within the field measurement margin of error.")
validation

<div class="alert alert-warning">
Validation Method 1: success for a baseline distance bin occurs when the percentage of residuals within the requirement success threshold is >0.8
</div>

### 4.4.4 Save Validation results

In [ ]:
# Save Method 1 results to file
run_date = dt.now().strftime('%Y%m%dT%H%M%S')
save_fldr = f"{run_date}-Permafrost-Method1"
# save_dir = os.path.join(mintpy_dir, save_fldr)
save_dir = str(Path.home()/'stash') #set your own location

validation_fig_method1 = fig
validation_table_method1 = validation

save_params = {
    'save_dir': save_dir,
    'run_date': run_date,
    'requirement': "Permafrost",
    'site': site,
    'method': "1",
    'sitedata': site_info,
    'gnss_insar_figs': [],
    'validation_figs': [validation_fig_method1],
    'validation_table': validation_table_method1,
    'ts_functions': None,
}
save_results(**save_params)

<a id='permafrost_insar_validate'></a>
# 5. Validation Method 2: InSAR-only structure fucntions

The NISAR permafrost validation requirement states that at least 80% of the time, the difference in surface displacement for two given points over 90 days should be no greater than $4(1+\sqrt{\text{L}})$ mm for points of L km apart, with $\text{L} \leq 50$ km.

The surface deformation across tundra permafrost will vary spatially due to multiple mechanisms, including soil type, excess surface ice, and local topography and climate. Variability occurs at spatial scales from centimeters to tens of kilometers. Nonetheless, we will evaluate the differences in surface displacement relative to the validation requirement using the InSAR signal alone over this terrain.

In [ ]:
#Permafrost requirements:
dist_th = np.linspace(0.1,50,100)  # distances for evaluation (km)
acpt_error = 4*(1+np.sqrt(dist_th))  # permafrost threshold in mm, for dist in km
acpt_error_cm = acpt_error/10.  #cm

In [ ]:
#show displacements at end of the timeseries
if 'timeseries_filename' not in locals():
    timeseries_filename = 'timeseries.h5'

with h5py.File(Path(mintpy_dir)/timeseries_filename, 'r') as f:
    dispmap = np.array(f['timeseries'][-1])*100

vmax = np.nanmax(np.abs(dispmap))
plt.figure()
plt.imshow(dispmap,vmin=-vmax,vmax=vmax,cmap='RdBu')
plt.colorbar(label='96 day displacement (cm)')

<a id='permafrost_insar_structure'></a>
##  5.1 Use structure functions to identify pixel pairs

We sample a large set of pixel pairs to compare relative displacements with. We use a water mask from the Hyp3 processing to mask out locations to not use. Information about how that water mask is developed can [be found here](https://storymaps.arcgis.com/stories/485916be1b1d46889aa436794b5633cb).

In [ ]:
__,atrib = readfile.read(Path(mintpy_dir)/timeseries_filename)

X0,Y0 = load_geo_utm(atrib)
X0_2d,Y0_2d = np.meshgrid(X0,Y0)

M2dist = []; rel_measure = []

tsmap = dispmap
tsmap[tsmap==0]=np.nan

# wmask = pu.get_watermask(Path(gunw_dir))
with h5py.File(Path(mintpy_dir)/'waterMask.h5') as data:
    wmask = data['waterMask'][:]
tsmap[wmask==0]=np.nan

#deramping will remove linear spatial trends in the displacement data.
#Currently disabled
dist_i, rel_measure_i = samp_pair(X0_2d,Y0_2d,tsmap,num_samples=1000000,deramp=False)

M2dist.append(dist_i)             #distance of pair, in m
rel_measure.append(rel_measure_i) #relative displacement of pair, in cm 
    
M2dist,rel_measure = np.array(M2dist),np.array(rel_measure)

#use only pixel pairs within 50 km
rel_measure = rel_measure[M2dist<=50e3]
M2dist=M2dist[M2dist<=50e3]

M2km = [i/1e3 for i in M2dist]  #convert distance to km
rmcm = [i for i in rel_measure] #relative measure in cm

fig, ax = plt.subplots(figsize=[9, 5.5])
img1 = ax.hist(M2km, bins=100)
ax.set_title(f"Histogram of distance")
ax.set_xlabel(r'Distance ($km$)')
ax.set_ylabel('Frequency')

fig, ax = plt.subplots(figsize=[9, 5.5])
img1 = ax.hist(rmcm, bins=100)
ax.set_title(f"Histogram of Relative Measurement")
ax.set_xlabel(r'Relative Measurement ($cm$)')
ax.set_ylabel('Frequency')

In [ ]:
start_date = dt.strptime(atrib['START_DATE'], '%Y%m%d')
end_date = dt.strptime(atrib['END_DATE'], '%Y%m%d')
start_str = start_date.strftime('%Y-%m-%d')
end_str = end_date.strftime('%Y-%m-%d')

# Define requirement
permafrost_threshold_rqmt = 4  # mm/yr
permafrost_distance_rqmt = [0.1, 50.0]  # km

# Validation parameters
n_bins = 10
# threshold = 0.683  
threshold = 0.8

pair_distance = np.array([i/1e3 for i in dist_i])
pair_difference = np.array([i*10 for i in rel_measure_i])

In [ ]:
validation_table,fig = display_permafrost_validation(pair_distance, # binned distance for point
                        pair_difference, # binned relative velocities mm/yr
                        site,
                        start_str,
                        end_str,
                        requirement=permafrost_threshold_rqmt,
                        req_dist_fcn=True,
                        distance_rqmt = permafrost_distance_rqmt,  # [0.1, 50] km
                        n_bins=n_bins,                  # number of bins, to collect statistics 
                        threshold=threshold,             
                        sensor='Sentinel-1',            # sensor that is validated, Sentinel-1 or NISAR
                        validation_type='permafrost',
                        validation_data='Field meas')   

In [ ]:
req_dist_fcn = True
reqval = 4
pair_req_met = np.array(pair_difference < reqval*(1+float(req_dist_fcn)*
                                                  np.sqrt(pair_distance)))
pair_req_met[np.isnan(pair_difference)]=np.nan
df = pd.DataFrame(np.vstack([pair_distance,
                            pair_difference,
                            pair_req_met]).T,
                            columns=['distance', 'double_diff','req_met'])

# remove nans
df_nonan = df.dropna(subset=['double_diff'])
bins = np.linspace(*permafrost_distance_rqmt, num=n_bins+1)
bin_centers = (bins[:-1] + bins[1:]) / 2
binned_df = df_nonan.groupby(pd.cut(df_nonan['distance'], bins),
                            observed=False)[['req_met']]

# get binned validation table 
bin_req = reqval*(1+float(req_dist_fcn)*np.sqrt(bin_centers))
validation = pd.DataFrame([])
validation['total_count[#]'] = binned_df.apply(lambda x: np.ma.masked_invalid(x)
    .count())
validation['passed_req.[#]'] = binned_df.apply(lambda x: np.count_nonzero(x))

# Add total at the end
validation = pd.concat([validation, pd.DataFrame(validation.sum(axis=0)).T])
validation['passed_pc'] = validation['passed_req.[#]'] / validation['total_count[#]']
validation['success_fail'] = validation['passed_pc'] > threshold
validation.index.name = 'distance[km]'
# Rename last row
validation.rename({validation.iloc[-1].name:'Total'}, inplace=True)
if validation.loc[validation.index[-1]]['success_fail']:
    print("This displacement dataset passes the requirement.")
else:
    print("This displacement dataset does not pass the requirement.")
validation

<div class="alert alert-warning">
Validation Method 1: success for a baseline distance bin occurs when the percentage of residuals within the requirement success threshold is >0.8
</div>

In [ ]:
# Save Method 2 results to file
run_date = dt.now().strftime('%Y%m%dT%H%M%S')
save_fldr = f"{run_date}-Permafrost-Method2"
save_dir = str(Path.home()/'stash') #set your own location

validation_fig_method2 = fig
validation_table_method2 = validation

save_params = {
    'save_dir': save_dir,
    'run_date': run_date,
    'requirement': "Permafrost",
    'site': site,
    'method': "2",
    'sitedata': site_info,
    'gnss_insar_figs': [],
    'validation_figs': [validation_fig_method2],
    'validation_table': validation_table_method2,
    'ts_functions': None,
}
save_results(**save_params)

<a id='permafrost_references'></a>
### References

Zwieback, Simon, and Franz J. Meyer. "Top-of-permafrost ground ice indicated by remotely sensed late-season subsidence." *The Cryosphere* 15 (2021): 2041-2055.